In [11]:
# add noise independently to each attribute
# use same epsilon and delta 
# use binary mechanism (https://diffprivlib.readthedocs.io/en/latest/modules/mechanisms.html#binary-mechanism) for sex
# use geometric mechanism (https://diffprivlib.readthedocs.io/en/latest/modules/mechanisms.html#geometric-mechanisms) for age
# use binary mechanism (https://diffprivlib.readthedocs.io/en/latest/modules/mechanisms.html#binary-mechanism) to select real value or representative random value for race

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from diffprivlib import tools as dp
from diffprivlib.mechanisms import Binary
from diffprivlib.mechanisms import Geometric
import pandas as pd
import random

#from all_us distribution
real_order = [58, 59, 60, 52, 54, 51, 48, 32, 42, 38, 36, 40, 34, 33, 43, 23, 16, 41, 29, 24, 15, 22, 28, 9, 21, 11, 4, 8, 6, 3, 1, 0, 5, 2, 10, 7, 14, 19, 12, 18, 17, 20, 13, 25, 27, 47, 31, 30, 26, 35, 56, 44, 37, 50, 45, 39, 57, 62, 46, 49, 55, 53, 61]

races = ['Total!!Population of one race!!White alone',
       'Total!!Population of one race!!Black or African American alone',
       'Total!!Population of one race!!American Indian and Alaska Native alone',
       'Total!!Population of one race!!Asian alone',
       'Total!!Population of one race!!Native Hawaiian and Other Pacific Islander alone',
       'Total!!Population of one race!!Some Other Race alone',
       'Total!!Two or More Races!!Population of two races!!White; Black or African American',
       'Total!!Two or More Races!!Population of two races!!White; American Indian and Alaska Native',
       'Total!!Two or More Races!!Population of two races!!White; Asian',
       'Total!!Two or More Races!!Population of two races!!White; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of two races!!White; Some Other Race',
       'Total!!Two or More Races!!Population of two races!!Black or African American; American Indian and Alaska Native',
       'Total!!Two or More Races!!Population of two races!!Black or African American; Asian',
       'Total!!Two or More Races!!Population of two races!!Black or African American; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of two races!!Black or African American; Some Other Race',
       'Total!!Two or More Races!!Population of two races!!American Indian and Alaska Native; Asian',
       'Total!!Two or More Races!!Population of two races!!American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of two races!!American Indian and Alaska Native; Some Other Race',
       'Total!!Two or More Races!!Population of two races!!Asian; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of two races!!Asian; Some Other Race',
       'Total!!Two or More Races!!Population of two races!!Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of three races!!White; Black or African American; American Indian and Alaska Native',
       'Total!!Two or More Races!!Population of three races!!White; Black or African American; Asian',
       'Total!!Two or More Races!!Population of three races!!White; Black or African American; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of three races!!White; Black or African American; Some Other Race',
       'Total!!Two or More Races!!Population of three races!!White; American Indian and Alaska Native; Asian',
       'Total!!Two or More Races!!Population of three races!!White; American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of three races!!White; American Indian and Alaska Native; Some Other Race',
       'Total!!Two or More Races!!Population of three races!!White; Asian; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of three races!!White; Asian; Some Other Race',
       'Total!!Two or More Races!!Population of three races!!White; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of three races!!Black or African American; American Indian and Alaska Native; Asian',
       'Total!!Two or More Races!!Population of three races!!Black or African American; American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of three races!!Black or African American; American Indian and Alaska Native; Some Other Race',
       'Total!!Two or More Races!!Population of three races!!Black or African American; Asian; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of three races!!Black or African American; Asian; Some Other Race',
       'Total!!Two or More Races!!Population of three races!!Black or African American; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of three races!!American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of three races!!American Indian and Alaska Native; Asian; Some Other Race',
       'Total!!Two or More Races!!Population of three races!!American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of three races!!Asian; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of four races!!White; Black or African American; American Indian and Alaska Native; Asian',
       'Total!!Two or More Races!!Population of four races!!White; Black or African American; American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of four races!!White; Black or African American; American Indian and Alaska Native; Some Other Race',
       'Total!!Two or More Races!!Population of four races!!White; Black or African American; Asian; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of four races!!White; Black or African American; Asian; Some Other Race',
       'Total!!Two or More Races!!Population of four races!!White; Black or African American; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of four races!!White; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of four races!!White; American Indian and Alaska Native; Asian; Some Other Race',
       'Total!!Two or More Races!!Population of four races!!White; American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of four races!!White; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of four races!!Black or African American; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of four races!!Black or African American; American Indian and Alaska Native; Asian; Some Other Race',
       'Total!!Two or More Races!!Population of four races!!Black or African American; American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of four races!!Black or African American; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of four races!!American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of five races!!White; Black or African American; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of five races!!White; Black or African American; American Indian and Alaska Native; Asian; Some Other Race',
       'Total!!Two or More Races!!Population of five races!!White; Black or African American; American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of five races!!White; Black or African American; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of five races!!White; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of five races!!Black or African American; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of six races!!White; Black or African American; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race']

race_dict1 = {}
race_dict2 = {}

for i in range(len(races)):
    race_dict1[i] = races[i]
    race_dict2[races[i]] = i

print(race_dict1)
tenuredict = {0: 'Own', 1:'Rent'}
tenuredict2 = {'Own': 0, 'Rent':1}

{0: 'Total!!Population of one race!!White alone', 1: 'Total!!Population of one race!!Black or African American alone', 2: 'Total!!Population of one race!!American Indian and Alaska Native alone', 3: 'Total!!Population of one race!!Asian alone', 4: 'Total!!Population of one race!!Native Hawaiian and Other Pacific Islander alone', 5: 'Total!!Population of one race!!Some Other Race alone', 6: 'Total!!Two or More Races!!Population of two races!!White; Black or African American', 7: 'Total!!Two or More Races!!Population of two races!!White; American Indian and Alaska Native', 8: 'Total!!Two or More Races!!Population of two races!!White; Asian', 9: 'Total!!Two or More Races!!Population of two races!!White; Native Hawaiian and Other Pacific Islander', 10: 'Total!!Two or More Races!!Population of two races!!White; Some Other Race', 11: 'Total!!Two or More Races!!Population of two races!!Black or African American; American Indian and Alaska Native', 12: 'Total!!Two or More Races!!Population of 

In [3]:
##########################################################################################################################
#CREATE BLOCK DATA
#input: csv containing block data 
#output: dataframe containing block data for age, sex, race, etc
##########################################################################################################################

def create_block_data(path):
    df = pd.read_csv(path)
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    return df

# df = create_block_data('../homemade_data/almeda.csv')
# orig_df = df.copy()

In [4]:
##########################################################################################################################
#MAKE DIFFERENTIALLY PRIVATE
#input: 1. dataframe dframe containing block data 2. race_data containing representative race vals 3. epsilon val (float)
#output: differentially private representation of dataframe df
##########################################################################################################################

def make_differentially_private(dft,  ep):
    cs = []
    ds = []
    es = []
    fs = []
    gs = []
    hs = []
    
    dframe = dft.copy()
    b_sex = Binary(epsilon=ep, value0="m", value1="f")
    geo_age = Geometric(epsilon=ep, sensitivity=115)
#     b2 = Binary(epsilon=ep, value0="true value", value1="fake value") I don't think we use this -Miranda
    geo_race = Geometric(epsilon=ep, sensitivity=62)
    geo_size = Geometric(epsilon=ep, sensitivity=3)
    b_tenure = Binary(epsilon=ep, value0="Own", value1="Rent")
    for (columnName, columnData) in dframe.iteritems():
        if columnName == "sex":
            i = 0
            for c in columnData.values:          
                d = b_sex.randomise(c)
                dframe.at[i,'sex'] = d
                i +=1
                cs.append(c)
                ds.append(d)
                
        if columnName == "age":
            i = 0
            for e in columnData.values:
                f = geo_age.randomise(int(e))
                dframe.at[i,'age'] = f
                i +=1
                es.append(e)
                fs.append(f)
        if columnName == "race":
            i = 0
            for g in columnData.values:
                idx = real_order.index(race_dict2[g])                
                blah = geo_race.randomise(idx)%63
                h = race_dict1[real_order[blah]]
                print(idx, blah)
                dframe.at[i, 'race'] = h
                i +=1
                
        if columnName == "household_size":
            i = 0
            for g in columnData.values:
                h = geo_size.randomise(g)%4 +1
                dframe.at[i, 'household_size'] = h
                i +=1
                
        if columnName == "household_tenure":
            i = 0
            for g in columnData.values:
#                 h = tenuredict[geo.randomise(tenuredict2[g])%2]
                h = b_tenure.randomise(g)
                dframe.at[i, 'household_tenure'] = h
                i +=1

    return dframe

        #testing




In [8]:
# Make dprun2 with one dataset per swap rate (for testing purposes)
counties = ['alameda', 'armstrong', 'cibola', 'fayette', 'grandforks', 'hawaii', 'jefferson', 'nantucket', 'washington']
counties = ['alameda']
for county in counties:
    df_first = create_block_data('../homemade_data/'+county+'.csv')
    orig_df = df_first.copy()
    csv_orig_data = df_first.to_csv('dp_runs/'+county+'/orig_data.csv', index = True)

    epsilon = np.arange(1, 7, 1, float)

    for e in epsilon:
        print(str(e)[0:4])
        df = df_first.copy()
        dframe = make_differentially_private(df, e)
#         filename = "dp_runs/"+county+"/dprun_"+str(e)[0:4]+".csv"
        filename = "dp_runs/"+county+"/dprun_"+str(e)[0:4]+".csv"
        csv_orig_data = dframe.to_csv(filename, index = True)

1.0
31 40
31 23
31 13
31 1
31 42
31 62
31 57
31 41
31 19
31 35
31 22
31 58
31 59
31 9
31 37
31 53
31 57
31 35
31 0
31 13
31 39
31 10
31 50
31 27
31 16
31 51
31 56
31 1
31 31
31 37
31 4
31 30
31 49
31 30
31 15
31 51
31 57
31 5
31 30
31 11
31 7
31 0
31 10
31 7
31 17
31 33
31 52
31 27
31 38
31 52
31 24
31 7
31 21
31 61
31 40
31 4
31 9
31 58
31 6
31 45
31 26
31 9
31 56
31 30
31 20
31 44
31 12
31 10
31 12
31 0
31 15
31 43
31 11
31 34
31 54
31 61
31 55
31 38
31 52
31 60
31 11
31 57
31 40
31 62
31 13
31 59
31 4
31 57
31 43
31 46
31 22
31 4
31 32
31 46
31 6
31 41
31 24
31 5
31 34
31 25
31 11
31 24
31 34
31 57
31 33
31 11
31 34
31 2
31 29
31 51
31 43
31 53
31 18
31 37
31 0
31 39
31 32
31 27
31 34
31 8
31 59
31 34
31 48
31 21
31 37
31 62
31 24
31 47
31 55
31 60
31 13
31 53
31 20
31 57
31 17
31 7
31 21
31 23
31 28
31 54
31 60
31 41
31 62
31 36
31 47
31 25
31 24
31 4
31 16
31 56
31 20
31 11
31 30
31 4
31 35
31 7
31 3
31 12
31 59
31 1
31 16
31 51
31 3
31 12
31 11
31 1
31 13
31 0
31 51
31 23
31 48
3

2.0
31 26
31 6
31 12
31 8
31 8
31 11
31 36
31 34
31 50
31 12
31 55
31 7
31 5
31 37
31 6
31 36
31 25
31 1
31 33
31 9
31 21
31 61
31 37
31 43
31 39
31 35
31 0
31 24
31 39
31 46
31 9
31 19
31 23
31 1
31 8
31 46
31 2
31 4
31 60
31 40
31 62
31 12
31 29
31 32
31 55
31 5
31 28
31 5
31 30
31 62
31 22
31 13
31 45
31 19
31 50
31 26
31 50
31 22
31 19
31 54
31 24
31 49
31 31
31 0
31 35
31 62
31 7
31 34
31 7
31 29
31 17
31 32
31 14
31 9
31 41
31 60
31 12
31 43
31 21
31 28
31 8
31 39
31 30
31 26
31 40
31 28
31 33
31 11
31 4
31 35
31 44
31 10
31 11
31 18
31 45
31 30
31 53
31 46
31 36
31 28
31 39
31 35
31 40
31 34
31 39
31 24
31 18
31 27
31 37
31 5
31 11
31 27
31 57
31 33
31 33
31 55
31 37
31 31
31 22
31 45
31 33
31 30
31 20
31 61
31 31
31 37
31 34
31 15
31 27
31 32
31 40
31 7
31 28
31 59
31 32
31 53
31 34
31 14
31 3
31 13
31 59
31 39
31 36
31 26
31 25
31 53
31 21
31 16
31 32
31 21
31 18
31 19
31 19
31 55
31 42
31 21
31 55
31 9
31 16
31 61
31 39
31 61
31 61
31 46
31 13
31 42
31 29
31 16
31 38
31 29
31

31 31
31 29
31 55
31 37
31 30
31 6
31 3
31 21
31 54
31 43
31 42
31 24
31 6
31 27
31 30
31 40
31 29
31 8
31 4
31 46
31 46
31 47
31 43
31 54
31 36
31 19
31 24
31 9
31 39
31 47
31 40
31 27
31 7
31 23
31 28
31 34
31 4
31 36
31 33
31 59
31 27
31 45
31 53
31 6
31 47
31 14
31 4
31 11
31 16
31 29
31 44
31 29
31 61
31 59
31 32
31 16
31 52
31 21
31 34
31 20
31 33
31 42
31 46
31 51
31 28
31 52
31 27
31 40
31 62
31 26
31 49
31 15
31 41
31 35
31 9
31 31
31 58
31 41
31 51
31 20
31 18
31 30
31 19
31 22
31 35
31 13
31 30
31 36
31 0
31 15
31 55
31 27
31 13
31 24
31 19
31 48
31 45
31 26
31 26
31 44
31 56
31 32
31 19
31 45
31 34
31 7
31 49
31 48
31 26
31 19
31 17
31 35
31 11
31 5
31 46
31 36
31 35
31 60
31 38
31 48
31 7
31 37
31 2
31 40
31 18
31 30
31 57
31 56
31 13
31 50
31 18
31 23
31 1
31 49
31 18
31 1
31 4
31 16
31 32
31 29
31 37
31 10
31 32
31 20
31 4
31 47
31 28
31 39
31 51
31 17
31 22
31 51
31 55
31 23
31 28
31 22
31 30
31 51
31 36
31 30
31 22
31 30
31 54
31 42
31 24
31 18
31 46
31 41
31 42
31 48


4.0
31 12
31 21
31 50
31 6
31 7
31 43
31 15
31 30
31 57
31 56
31 26
31 57
31 36
31 14
31 12
31 60
31 25
31 31
31 17
31 19
31 45
31 1
31 29
31 53
31 28
31 40
31 17
31 31
31 37
31 51
31 27
31 29
31 60
31 27
31 31
31 17
31 21
31 36
31 25
31 26
31 23
31 13
31 58
31 33
31 28
31 59
31 14
31 15
31 13
31 39
31 53
31 23
31 38
31 36
31 22
31 3
31 56
31 32
31 33
31 34
31 51
31 46
31 36
31 16
31 14
31 39
31 18
31 24
31 12
31 48
31 0
31 35
31 33
31 48
31 34
31 36
31 46
31 45
31 21
31 21
31 23
31 19
31 49
31 9
31 6
31 48
31 22
31 46
31 42
31 44
31 39
31 11
31 10
31 23
31 29
31 16
31 40
31 37
31 13
31 11
31 11
31 27
31 2
31 14
31 18
31 37
31 24
31 3
31 35
31 28
31 13
31 25
31 26
31 26
31 37
31 28
31 3
31 33
31 31
31 14
31 31
31 31
31 58
31 31
31 52
31 15
31 44
31 60
31 32
31 27
31 59
31 30
31 41
31 48
31 33
31 15
31 43
31 43
31 24
31 52
31 34
31 11
31 28
31 27
31 30
31 3
31 49
31 44
31 13
31 41
31 39
31 61
31 41
31 46
31 55
31 52
31 27
31 9
31 24
31 62
31 28
31 38
31 25
31 24
31 31
31 35
31 31
31 26


5.0
31 20
31 50
31 62
31 31
31 21
31 34
31 0
31 9
31 25
31 17
31 32
31 4
31 8
31 31
31 31
31 34
31 57
31 58
31 57
31 22
31 15
31 28
31 7
31 36
31 38
31 9
31 27
31 17
31 35
31 61
31 27
31 30
31 1
31 31
31 27
31 29
31 27
31 32
31 9
31 29
31 31
31 10
31 10
31 25
31 23
31 33
31 24
31 45
31 43
31 36
31 11
31 37
31 36
31 50
31 37
31 50
31 30
31 22
31 39
31 44
31 36
31 49
31 33
31 37
31 1
31 14
31 38
31 49
31 13
31 36
31 28
31 19
31 26
31 34
31 0
31 31
31 28
31 58
31 21
31 10
31 29
31 35
31 32
31 57
31 17
31 33
31 39
31 54
31 27
31 8
31 22
31 23
31 37
31 23
31 13
31 44
31 17
31 29
31 26
31 54
31 17
31 32
31 31
31 24
31 45
31 29
31 7
31 33
31 21
31 19
31 30
31 31
31 18
31 34
31 27
31 7
31 39
31 37
31 33
31 32
31 9
31 15
31 38
31 53
31 29
31 44
31 39
31 38
31 27
31 20
31 17
31 41
31 27
31 41
31 39
31 31
31 26
31 40
31 32
31 43
31 30
31 31
31 50
31 60
31 43
31 27
31 29
31 44
31 31
31 17
31 22
31 38
31 26
31 49
31 34
31 32
31 28
31 31
31 54
31 31
31 32
31 35
31 38
31 26
31 25
31 23
31 37
31 28
31

6.0
31 33
31 30
31 35
31 35
31 30
31 33
31 34
31 12
31 25
31 24
31 30
31 24
31 32
31 20
31 25
31 20
31 34
31 24
31 31
31 26
31 35
31 34
31 31
31 16
31 58
31 30
31 35
31 43
31 23
31 38
31 24
31 47
31 51
31 36
31 58
31 49
31 45
31 31
31 28
31 39
31 23
31 33
31 44
31 32
31 31
31 23
31 25
31 32
31 34
31 32
31 57
31 31
31 30
31 31
31 38
31 11
31 23
31 3
31 29
31 27
31 25
31 25
31 25
31 15
31 58
31 37
31 4
31 34
31 25
31 62
31 30
31 30
31 59
31 38
31 23
31 32
31 19
31 34
31 33
31 47
31 32
31 27
31 40
31 38
31 41
31 36
31 28
31 30
31 18
31 32
31 31
31 54
31 28
31 2
31 19
31 14
31 35
31 48
31 35
31 28
31 29
31 43
31 37
31 21
31 25
31 22
31 39
31 41
31 31
31 52
31 33
31 19
31 13
31 27
31 19
31 32
31 60
31 17
31 6
31 25
31 15
31 29
31 23
31 23
31 40
31 39
31 34
31 27
31 32
31 46
31 44
31 29
31 10
31 4
31 16
31 34
31 39
31 23
31 28
31 32
31 21
31 31
31 22
31 54
31 21
31 28
31 31
31 44
31 48
31 57
31 31
31 24
31 31
31 27
31 19
31 24
31 35
31 19
31 30
31 26
31 38
31 29
31 34
31 17
31 34
31 2
31 37


FileNotFoundError: [Errno 2] No such file or directory: 'dp_runs3/armstrong/orig_data.csv'

In [9]:
# Make dp_runs2 with 100 datasets per swap rate
# # Naming convention: dprun_{epsilon}_{number from 0 to 100}.csv
# counties = ['alameda', 'armstrong', 'cibola', 'fayette', 'grandforks', 'hawaii', 'jefferson', 'nantucket', 'washington']

# for county in counties:
#     df_first = create_block_data('../homemade_data/'+county+'.csv')
#     orig_df = df_first.copy()
#     csv_orig_data = df_first.to_csv('dp_runs/'+county+'/orig_data.csv', index = True)

#     epsilon = np.arange(1, 7, .1, float)

#     for e in epsilon:
#         print(str(e)[0:4])
#         for i in range(100):
#             df = df_first.copy()
#             dframe = make_differentially_private(df, e)
#     #         filename = "dp_runs/"+county+"/dprun_"+str(e)[0:4]+".csv"
#             filename = "dp_runs2/"+county+"/dprun_"+str(e)[0:4]+"_"+str(i)+".csv"
#             csv_orig_data = dframe.to_csv(filename, index = True)